In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from scipy.stats import norm

In [ ]:
model_name = '06'

In [ ]:
model_pth = '../results/rewards/gradient/sub-'+model_name+'/'+'rewards_30.npy'
model_rewards = np.load(model_pth)

In [ ]:
subject_rewards_1_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d1.csv'
subject_rewards_2_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d2.csv'
subject_rewards_3_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d3.csv'
all_subject_reward_pths = [subject_rewards_1_pth, subject_rewards_2_pth, subject_rewards_3_pth]

In [ ]:
subject_rewards = [pd.read_csv(pth, header=None).values for pth in all_subject_reward_pths]
subject_rewards = np.vstack(subject_rewards)

In [ ]:
def fit_gaussian_and_compute_nll(train_data, test_data):
    # Fit a Gaussian distribution to the training data
    mu, sigma = np.mean(train_data), np.std(train_data, ddof=1)  # Unbiased estimator
    
    # Compute the negative log-likelihood (NLL) for the test data
    nll = -np.sum(norm.logpdf(test_data, loc=mu, scale=sigma))
    
    return mu, sigma, nll

In [ ]:
fit_gaussian_and_compute_nll(subject_rewards, model_rewards[0, :, 0])

In [ ]:
nlls = np.zeros((len(model_rewards), np.shape(model_rewards)[2]))
for idx, model_reward in enumerate(model_rewards):
    for rep in range(np.shape(model_rewards)[2]):
        _, _, nll = fit_gaussian_and_compute_nll(subject_rewards, model_reward[:,  rep])
        nlls[idx, rep] = nll

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(nlls.T, labels=[f'Model {i+1}' for i in range(nlls.shape[0])])
plt.xlabel('Model')
plt.ylabel('Negative Log-Likelihood (NLL)')
plt.title('Box Plots of NLLs for Different Models')

fig_nanme = 'sub-'+model_name+'_nll_boxplot.pdf'
plt.savefig('../results/Imgs/sub-'+model_name+'/'+fig_nanme)
plt.show()